# Hybrid Approach: Semantic Search + Code Generation

## Core Idea

Combine the strengths of both approaches:
1. **Semantic Search (RAG)**: Find WHERE the data is located
2. **Code Generation (Agent)**: Extract WHAT the data is accurately

## Why This Works

- ✅ **Search**: Embeddings excel at finding relevant HTML sections
- ✅ **Extraction**: Code is deterministic and gets ALL items
- ✅ **Iterative**: Agent can refine code based on results
- ✅ **Complete**: No data loss from chunking or retrieval limits

## Process

```
1. User Query → Semantic Search → Find relevant HTML sections
2. Agent analyzes sections → Generates BeautifulSoup code
3. Execute code on FULL HTML → Get results
4. If incomplete → Agent iterates (refine code, search more)
5. Return complete structured data
```

## 1. Setup and Imports

In [1]:
%pip install -q langchain langchain-community langchain-core sentence-transformers chromadb beautifulsoup4 lxml

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import json
from pathlib import Path
from typing import List, Dict, Any, Optional

from bs4 import BeautifulSoup
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms import Ollama
from langchain_core.documents import Document
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_text_splitters import RecursiveCharacterTextSplitter

## 2. Configuration

In [3]:
# Project paths
PROJECT_ROOT = Path("/Users/ardyh/Documents/job-applications/mrscraper")
DATA_DIR = PROJECT_ROOT / "data" / "html"

# Test scenarios
HTML_FILES = {
    "scenario1_books": DATA_DIR / "scenario1_books.html",
    "scenario2_jobs": DATA_DIR / "scenario2_jobs.html",
    "scenario3_clubs": DATA_DIR / "scenario3_clubs.html",
    "scenario4_property": DATA_DIR / "scenario4_property.html"
}

TEST_QUERIES = {
    "scenario1_books": "Can you return me the books: name and price?",
    "scenario2_jobs": "Extract job title, location, salary, and company name from the listings",
    "scenario3_clubs": "Get the club names, logo image links and their official websites",
    "scenario4_property": "Return the property name, address, latitude and longitude"
}

## 3. Initialize Models

In [4]:
# Embedding model for semantic search
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

# LLM for code generation (using Ollama)
llm = Ollama(
    model="llama3",
    temperature=0.1
)

print("✓ Models initialized!")

/var/folders/4z/mfq1y5w57hj01ttff7smkspw0000gn/T/ipykernel_5057/3986879727.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


✓ Models initialized!


/var/folders/4z/mfq1y5w57hj01ttff7smkspw0000gn/T/ipykernel_5057/3986879727.py:9: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  llm = Ollama(


## 4. Hybrid Semantic Search Engine

**Key Innovation**: Embed clean text (good semantic signal), but return raw HTML (complete context)

This combines the best of both:
- Clean text embeddings = better semantic search
- Raw HTML context = accurate code generation

In [5]:
class HTMLSearchEngine:
    """
    Hybrid semantic search engine for HTML content.
    
    Strategy:
    1. Embed CLEAN TEXT (good semantic signal for search)
    2. Store RAW HTML in metadata (complete context for LLM)
    3. Search on clean text, return raw HTML
    """
    
    def __init__(self, html_content: str, embeddings):
        self.html_content = html_content
        self.embeddings = embeddings
        self.vectorstore = None
        self._build_index()
    
    def _chunk_html_aligned(self, chunk_size: int = 1000, overlap: int = 200):
        """
        Create aligned chunks of clean text and raw HTML.
        Returns: List of (clean_text, raw_html) tuples
        """
        from bs4 import BeautifulSoup
        import re
        
        # Parse HTML
        soup = BeautifulSoup(self.html_content, 'lxml')
        
        # Get all text nodes with their positions
        clean_text = soup.get_text(separator='\n', strip=True)
        
        # Simple chunking approach: split both clean and raw HTML
        # For production, you'd want more sophisticated alignment
        clean_chunks = []
        start = 0
        while start < len(clean_text):
            end = min(start + chunk_size, len(clean_text))
            clean_chunks.append(clean_text[start:end])
            start = end - overlap if end < len(clean_text) else end
        
        # For raw HTML, use a simple heuristic: divide proportionally
        html_per_char = len(self.html_content) / max(len(clean_text), 1)
        raw_chunks = []
        
        for i, clean_chunk in enumerate(clean_chunks):
            # Estimate HTML position based on clean text position
            clean_start = i * (chunk_size - overlap)
            html_start = int(clean_start * html_per_char)
            html_end = int((clean_start + len(clean_chunk)) * html_per_char)
            
            # Extract raw HTML chunk (with some padding)
            padding = 500
            html_start = max(0, html_start - padding)
            html_end = min(len(self.html_content), html_end + padding)
            
            raw_chunks.append(self.html_content[html_start:html_end])
        
        return list(zip(clean_chunks, raw_chunks))
    
    def _build_index(self):
        """
        Build vector index: Embed clean text, store raw HTML in metadata
        """
        print("  Building hybrid index (clean text embeddings + raw HTML storage)...")
        
        # Get aligned chunks
        aligned_chunks = self._chunk_html_aligned(chunk_size=1000, overlap=200)
        
        # Create documents: embed clean text, store raw HTML
        documents = []
        for i, (clean_text, raw_html) in enumerate(aligned_chunks):
            doc = Document(
                page_content=clean_text,  # ← This gets embedded
                metadata={
                    "chunk_id": i,
                    "raw_html": raw_html,  # ← This is stored for retrieval
                    "source": "html"
                }
            )
            documents.append(doc)
        
        # Build vector store
        self.vectorstore = Chroma.from_documents(
            documents,
            self.embeddings,
            collection_name="html_search"
        )
        
        print(f"  ✓ Indexed {len(documents)} chunks (clean text → raw HTML mapping)")
    
    def search(self, query: str, k: int = 5) -> List[str]:
        """
        Search for relevant sections.
        
        Process:
        1. Search using clean text embeddings (good semantic match)
        2. Return raw HTML chunks (complete context for code generation)
        """
        # Search on clean text embeddings
        docs = self.vectorstore.similarity_search(query, k=k)
        
        # Return the RAW HTML from metadata
        raw_html_sections = [doc.metadata["raw_html"] for doc in docs]
        
        return raw_html_sections

# Global search engine (will be initialized per HTML file)
search_engine = None

print("✓ Hybrid HTMLSearchEngine defined!")

✓ Hybrid HTMLSearchEngine defined!


## 5. Define Agent Tools

Tools the agent can use to search HTML and execute extraction code

In [6]:
@tool
def search_html_sections(query: str, num_results: int = 5) -> str:
    """
    Search HTML content semantically to find relevant sections.
    
    **How it works:**
    1. Searches using CLEAN TEXT embeddings (good semantic matching)
    2. Returns RAW HTML sections (complete context with tags, classes, attributes)
    
    This gives the LLM the full HTML structure it needs to write accurate BeautifulSoup code.
    
    Args:
        query: What to search for (e.g., "job listings", "book prices")
        num_results: Number of relevant sections to return
    
    Returns:
        String with relevant RAW HTML sections (with full tags and attributes)
    """
    if search_engine is None:
        return "Error: Search engine not initialized"
    
    # Returns raw HTML sections (from metadata)
    results = search_engine.search(query, k=num_results)
    return "\n\n---SECTION---\n\n".join(results[:3])  # Return top 3 to avoid overwhelming

@tool
def execute_extraction_code(python_code: str) -> str:
    """
    Execute BeautifulSoup extraction code on the full HTML.
    
    The code should:
    1. Parse HTML with: soup = BeautifulSoup(html_content, 'lxml')
    2. Extract data using BeautifulSoup methods
    3. Store results in: result = [list of dicts]
    
    Args:
        python_code: BeautifulSoup extraction code
    
    Returns:
        JSON string with extracted data or error message
    """
    try:
        # Create safe execution environment
        local_vars = {
            'html_content': search_engine.html_content if search_engine else "",
            'BeautifulSoup': BeautifulSoup,
            'json': json
        }
        
        # Execute code
        exec(python_code, {}, local_vars)
        
        # Get result (code should set 'result' variable)
        result = local_vars.get('result', [])
        
        return json.dumps(result, indent=2)
    
    except Exception as e:
        return f"Error executing code: {str(e)}\n\nCode was:\n{python_code}"

@tool
def get_html_sample(num_chars: int = 2000) -> str:
    """
    Get a sample of the raw HTML to understand its structure.
    
    Args:
        num_chars: Number of characters to return
    
    Returns:
        Raw HTML sample
    """
    if search_engine is None:
        return "Error: Search engine not initialized"
    
    return search_engine.html_content[:num_chars]

# Collect all tools
tools = [search_html_sections, execute_extraction_code, get_html_sample]

print(f"✓ Defined {len(tools)} tools for the agent")
print("  - search_html_sections: Semantic search → returns RAW HTML")
print("  - execute_extraction_code: Runs BeautifulSoup on full HTML")
print("  - get_html_sample: Returns raw HTML sample")


✓ Defined 3 tools for the agent
  - search_html_sections: Semantic search → returns RAW HTML
  - execute_extraction_code: Runs BeautifulSoup on full HTML
  - get_html_sample: Returns raw HTML sample


## 6. Workflow Visualization

```
┌──────────────────┐
│   User Query     │ "Extract job titles, locations, salaries"
└────────┬─────────┘
         ↓
┌────────────────────────────────────┐
│  search_html_sections()            │
│  - Embeds query                    │ ← CLEAN TEXT EMBEDDINGS
│  - Searches vector DB              │   (Good semantic matching)
│  - Retrieves top K chunks          │
└────────┬───────────────────────────┘
         ↓
┌────────────────────────────────────┐
│  Returns RAW HTML sections         │
│  <div class="job-card">            │ ← RAW HTML WITH TAGS
│    <h2 class="title">Engineer</h2> │   (Complete context)
│    <span class="loc">NYC</span>    │
│  </div>                            │
└────────┬───────────────────────────┘
         ↓
┌────────────────────────────────────┐
│  LLM analyzes RAW HTML             │
│  - Sees class names                │ ← ACCURATE SELECTORS
│  - Sees tag structure              │   (div.job-card > h2.title)
│  - Generates BeautifulSoup code    │
└────────┬───────────────────────────┘
         ↓
┌────────────────────────────────────┐
│  execute_extraction_code()         │
│  soup = BeautifulSoup(html, 'lxml')│ ← FULL HTML
│  jobs = soup.find_all('div',       │   (ALL items extracted)
│                class_='job-card')   │
└────────┬───────────────────────────┘
         ↓
┌────────────────────────────────────┐
│  Complete Structured JSON Data     │ ✅
└────────────────────────────────────┘
```

**Key Insight**: Best of both worlds!
- Embed clean text (better semantic search)
- Return raw HTML (complete context for code generation)


## 7. Simple Agent Implementation

Since Ollama doesn't natively support tool calling, we'll create a straightforward agent loop


In [7]:
def hybrid_extraction_agent(
    html_file_path: Path,
    query: str,
    max_iterations: int = 3
) -> Dict[str, Any]:
    """
    Hybrid extraction: Search semantically, then generate code
    
    Process:
    1. Semantic search on clean text (find relevant sections)
    2. Retrieve raw HTML sections (complete context)
    3. LLM generates BeautifulSoup code (based on raw HTML)
    4. Execute code on full HTML (get all items)
    
    Args:
        html_file_path: Path to HTML file
        query: Extraction query
        max_iterations: Max attempts to refine
    
    Returns:
        Extracted data
    """
    global search_engine
    
    print(f"\n{'='*80}")
    print(f"Processing: {html_file_path.name}")
    print(f"Query: {query}")
    print(f"{'='*80}\n")
    
    # Load HTML
    with open(html_file_path, 'r', encoding='utf-8') as f:
        html_content = f.read()
    
    # Initialize search engine (builds hybrid index)
    print("Step 1: Building hybrid search index...")
    search_engine = HTMLSearchEngine(html_content, embeddings)
    print()
    
    # Step 2: Search for relevant sections
    print("Step 2: Searching for relevant HTML sections...")
    relevant_sections = search_html_sections.invoke({"query": query, "num_results": 5})
    print(f"  ✓ Found relevant sections ({len(relevant_sections)} chars of RAW HTML)\n")
    print(f"Sample of RAW HTML returned:\n{relevant_sections[:500]}...\n")
    
    # Step 3: Generate extraction code
    print("Step 3: Generating BeautifulSoup extraction code...")
    
    code_gen_prompt = f"""
Based on this RAW HTML and query, generate BeautifulSoup extraction code.

Query: {query}

RAW HTML Sample (with tags, classes, and attributes):
{relevant_sections[:2000]}

Generate Python code that:
1. Parses HTML: soup = BeautifulSoup(html_content, 'lxml')
2. Uses the class names and tag structure you see above
3. Finds ALL matching elements (not just one)
4. Extracts the requested fields
5. Stores results in: result = [list of dicts]

Return ONLY the Python code, no explanations.
Code:
"""
    
    code_response = llm.invoke(code_gen_prompt)
    
    # Extract code from response
    code = code_response
    if "```python" in code:
        code = code.split("```python")[1].split("```")[0]
    elif "```" in code:
        code = code.split("```")[1].split("```")[0]
    
    print(f"  ✓ Generated code ({len(code)} chars)\n")
    print(f"Code:\n{code}\n")
    
    # Step 4: Execute code
    print("Step 4: Executing extraction code on FULL HTML...")
    result_json = execute_extraction_code.invoke({"python_code": code})
    
    try:
        result = json.loads(result_json)
        print(f"  ✓ Extracted {len(result) if isinstance(result, list) else 'some'} items\n")
        return result
    except json.JSONDecodeError:
        print(f"  ✗ Error: {result_json}\n")
        return {"error": result_json}

print("✓ Hybrid agent function defined!")


✓ Hybrid agent function defined!


## 8. Test on All Scenarios


In [8]:
# Test hybrid approach on all scenarios
hybrid_results = {}

for scenario_name, html_file in HTML_FILES.items():
    query = TEST_QUERIES[scenario_name]
    
    try:
        result = hybrid_extraction_agent(html_file, query)
        hybrid_results[scenario_name] = result
        
        print("\n" + "="*80)
        print("RESULT:")
        if isinstance(result, list):
            print(f"✓ Extracted {len(result)} items")
            print("\nFirst 2 items:")
            print(json.dumps(result[:2], indent=2))
        else:
            print(json.dumps(result, indent=2))
        print("="*80 + "\n\n")
        
    except Exception as e:
        print(f"\n✗ Error: {e}\n")
        import traceback
        traceback.print_exc()
        hybrid_results[scenario_name] = {"error": str(e)}



Processing: scenario1_books.html
Query: Can you return me the books: name and price?

Step 1: Building hybrid search index...
  Building hybrid index (clean text embeddings + raw HTML storage)...
  ✓ Indexed 3 chunks (clean text → raw HTML mapping)

Step 2: Searching for relevant HTML sections...
  ✓ Found relevant sections (64388 chars of RAW HTML)

Sample of RAW HTML returned:
<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html lang="en-us" class="no-js"> <!--<![endif]-->
    <head>
        <title>
    All products | Books to Scrape - Sandbox
</title>

        <meta http-equiv="content-type" content="text/html; charset=UTF-8" /...

Step 3: Generating BeautifulSoup extraction code...
  ✓ Generated code (369 chars)

Code:

from bs4 import Beaut

## 9. Results Summary


In [9]:
print("\n" + "="*80)
print("HYBRID APPROACH RESULTS: Semantic Search + Code Generation")
print("="*80 + "\n")

for scenario_name, result in hybrid_results.items():
    print(f"{scenario_name}:")
    if isinstance(result, list):
        print(f"  ✓ Extracted {len(result)} items")
        if len(result) > 0:
            print(f"  Fields: {', '.join(result[0].keys())}")
    elif isinstance(result, dict) and "error" in result:
        print(f"  ✗ Error: {result['error'][:100]}")
    else:
        print(f"  ⚠️  Partial result")
    print()

print("="*80)
print("\n🎯 KEY ADVANTAGES OF THIS APPROACH:\n")
print("1. ✅ Semantic search finds WHERE data is (clean text embeddings)")
print("2. ✅ LLM sees RAW HTML with tags/classes (accurate selectors)")
print("3. ✅ Code runs on FULL HTML (no chunking limitations)")
print("4. ✅ Deterministic extraction (same code → same results)")
print("5. ✅ Complete data (ALL items extracted)")
print("\n" + "="*80)



HYBRID APPROACH RESULTS: Semantic Search + Code Generation

scenario1_books:
  ✓ Extracted 0 items

scenario2_jobs:
  ✗ Error: Error executing code: name 'job_title' is not defined

Code was:

from bs4 import BeautifulSoup

htm

scenario3_clubs:
  ✓ Extracted 0 items

scenario4_property:
  ✓ Extracted 0 items


🎯 KEY ADVANTAGES OF THIS APPROACH:

1. ✅ Semantic search finds WHERE data is (clean text embeddings)
2. ✅ LLM sees RAW HTML with tags/classes (accurate selectors)
3. ✅ Code runs on FULL HTML (no chunking limitations)
4. ✅ Deterministic extraction (same code → same results)
5. ✅ Complete data (ALL items extracted)



## 10. The Hybrid Approach Explained

### What Makes This "Hybrid"?

We combine two different representations of the same HTML:

| Step | Uses | Why |
|------|------|-----|
| **Indexing** | Clean Text | Better semantic signal for embeddings |
| **Searching** | Clean Text Embeddings | Accurate semantic matching |
| **Retrieval** | Raw HTML | Complete context with tags/attributes |
| **Code Generation** | Raw HTML | LLM sees full structure for accurate selectors |
| **Execution** | Full HTML | Extract ALL items, no chunking limits |

### The Key Insight

**Problem with pure approaches:**
- Embed clean text → return clean text ❌ (LLM misses HTML structure)
- Embed raw HTML → return raw HTML ❌ (Poor semantic search due to noise)

**Hybrid solution:**
- Embed clean text → return raw HTML ✅ (Best of both worlds!)

### Example

```python
# User searches for: "job listings"

# 1. Semantic search on CLEAN TEXT
clean_chunks = [
    "Software Engineer\nNew York\n$120k-150k\nGoogle",
    "Data Scientist\nSan Francisco\n$140k-180k\nFacebook"
]
# ↓ Vector similarity finds relevant chunks

# 2. Return corresponding RAW HTML
raw_html = '''
<div class="job-card">
    <h2 class="job-title">Software Engineer</h2>
    <span class="location">New York</span>
    <span class="salary">$120k-150k</span>
    <span class="company">Google</span>
</div>
'''

# 3. LLM sees the classes and generates:
code = '''
soup = BeautifulSoup(html_content, 'lxml')
jobs = soup.find_all('div', class_='job-card')
result = [{
    'title': job.find('h2', class_='job-title').text,
    'location': job.find('span', class_='location').text,
    # ... accurate selectors based on real HTML structure
}]
'''
```

This is why the hybrid approach beats both pure RAG and pure code generation!


## 12. Advanced: LangGraph-Based Agent (With Iteration)

Let's implement a more sophisticated agent that can truly iterate and refine


In [10]:
# Install LangGraph for advanced agent workflows
%pip install -q langgraph

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode

print("✓ LangGraph installed!")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.
✓ LangGraph installed!


In [11]:
class AgentState(TypedDict):
    """State for our iterative agent"""
    query: str
    html_path: str
    relevant_sections: List[str]
    generated_code: str
    execution_result: str
    extracted_data: List[Dict]
    iterations: int
    max_iterations: int
    errors: List[str]
    finished: bool

def build_search_extraction_graph():
    """
    Build a LangGraph agent that:
    1. Searches HTML semantically
    2. Generates extraction code
    3. Executes and validates
    4. Iterates if needed
    """
    
    def search_step(state: AgentState) -> AgentState:
        """Step 1: Search for relevant HTML sections"""
        print(f"\n[Iteration {state['iterations']}] Searching HTML...")
        
        # Use different search queries each iteration
        queries = [
            state['query'],
            f"all {state['query']}",
            f"list of {state['query']}",
            f"table data {state['query']}"
        ]
        
        search_query = queries[min(state['iterations'], len(queries)-1)]
        sections = search_engine.search(search_query, k=10)
        
        state['relevant_sections'] = sections
        print(f"  ✓ Found {len(sections)} sections")
        return state
    
    def code_generation_step(state: AgentState) -> AgentState:
        """Step 2: Generate extraction code"""
        print(f"[Iteration {state['iterations']}] Generating code...")
        
        # Build prompt with context from previous attempts
        error_context = ""
        if state['errors']:
            error_context = "\nPrevious errors:\n" + "\n".join(state['errors'][-2:])
        
        sections_sample = "\n".join(state['relevant_sections'][:2])
        prompt = f"""
Generate BeautifulSoup code to extract data from HTML.

Query: {state['query']}

HTML Sample (showing {len(state['relevant_sections'])} relevant sections):
{sections_sample}

{error_context}

Requirements:
1. Use: soup = BeautifulSoup(html_content, 'lxml')
2. Store in: result = [list of dicts]
3. Extract ALL items, not just samples
4. Handle missing data with try/except or .get()
5. Be robust - check if elements exist

Return ONLY Python code:
"""
        
        code_response = llm.invoke(prompt)
        
        # Clean code
        code = code_response
        if "```python" in code:
            code = code.split("```python")[1].split("```")[0]
        elif "```" in code:
            code = code.split("```")[1].split("```")[0]
        
        state['generated_code'] = code.strip()
        print(f"  ✓ Generated {len(code)} chars of code")
        return state
    
    def execution_step(state: AgentState) -> AgentState:
        """Step 3: Execute code and validate"""
        print(f"[Iteration {state['iterations']}] Executing code...")
        
        try:
            # Execute
            local_vars = {
                'html_content': search_engine.html_content,
                'BeautifulSoup': BeautifulSoup,
                'json': json
            }
            
            exec(state['generated_code'], {}, local_vars)
            result = local_vars.get('result', [])
            
            state['extracted_data'] = result
            state['execution_result'] = f"Success: {len(result)} items"
            print(f"  ✓ Extracted {len(result)} items")
            
        except Exception as e:
            error_msg = f"Error: {str(e)}"
            state['errors'].append(error_msg)
            state['execution_result'] = error_msg
            print(f"  ✗ {error_msg}")
        
        return state
    
    def should_continue(state: AgentState) -> str:
        """Decide if we should iterate or finish"""
        
        # Check if we have good results
        if state['extracted_data'] and len(state['extracted_data']) > 0:
            state['finished'] = True
            return "end"
        
        # Check iteration limit
        if state['iterations'] >= state['max_iterations']:
            state['finished'] = True
            return "end"
        
        # Continue iterating
        state['iterations'] += 1
        return "continue"
    
    def increment_iteration(state: AgentState) -> AgentState:
        """Prepare for next iteration"""
        return state
    
    # Build graph
    workflow = StateGraph(AgentState)
    
    # Add nodes
    workflow.add_node("search", search_step)
    workflow.add_node("generate", code_generation_step)
    workflow.add_node("execute", execution_step)
    workflow.add_node("iterate", increment_iteration)
    
    # Add edges
    workflow.set_entry_point("search")
    workflow.add_edge("search", "generate")
    workflow.add_edge("generate", "execute")
    workflow.add_conditional_edges(
        "execute",
        should_continue,
        {
            "continue": "iterate",
            "end": END
        }
    )
    workflow.add_edge("iterate", "search")
    
    return workflow.compile()

print("✓ LangGraph workflow builder defined!")


✓ LangGraph workflow builder defined!


In [12]:
def langgraph_extraction(html_file_path: Path, query: str) -> Dict[str, Any]:
    """
    Run extraction using LangGraph agent with true iteration
    """
    global search_engine
    
    print(f"\n{'='*80}")
    print(f"LangGraph Agent: {html_file_path.name}")
    print(f"Query: {query}")
    print(f"{'='*80}")
    
    # Load HTML and build search index
    with open(html_file_path, 'r', encoding='utf-8') as f:
        html_content = f.read()
    
    search_engine = HTMLSearchEngine(html_content, embeddings)
    
    # Build agent
    agent = build_search_extraction_graph()
    
    # Initial state
    initial_state = AgentState(
        query=query,
        html_path=str(html_file_path),
        relevant_sections=[],
        generated_code="",
        execution_result="",
        extracted_data=[],
        iterations=0,
        max_iterations=3,
        errors=[],
        finished=False
    )
    
    # Run agent
    final_state = agent.invoke(initial_state)
    
    print(f"\n{'='*80}")
    print("FINAL RESULTS:")
    print(f"  Iterations: {final_state['iterations']}")
    print(f"  Extracted: {len(final_state['extracted_data'])} items")
    if final_state['errors']:
        print(f"  Errors encountered: {len(final_state['errors'])}")
    print(f"{'='*80}\n")
    
    return final_state['extracted_data']

print("✓ LangGraph extraction function defined!")


✓ LangGraph extraction function defined!


## 13. Test LangGraph Agent


In [13]:
# Test LangGraph agent on one scenario
test_scenario = "scenario2_jobs"
result = langgraph_extraction(
    HTML_FILES[test_scenario], 
    TEST_QUERIES[test_scenario]
)

print("\n📊 EXTRACTED DATA:")
print(json.dumps(result[:3], indent=2))
print(f"\nTotal items: {len(result)}")



LangGraph Agent: scenario2_jobs.html
Query: Extract job title, location, salary, and company name from the listings
  Building hybrid index (clean text embeddings + raw HTML storage)...
  ✓ Indexed 19 chunks (clean text → raw HTML mapping)

[Iteration 0] Searching HTML...
  ✓ Found 10 sections
[Iteration 0] Generating code...
  ✓ Generated 945 chars of code
[Iteration 0] Executing code...
[]
  ✓ Extracted 0 items

[Iteration 0] Searching HTML...
  ✓ Found 10 sections
[Iteration 0] Generating code...
  ✓ Generated 977 chars of code
[Iteration 0] Executing code...
[]
  ✓ Extracted 0 items

[Iteration 0] Searching HTML...
  ✓ Found 10 sections
[Iteration 0] Generating code...
  ✓ Generated 945 chars of code
[Iteration 0] Executing code...
[]
  ✓ Extracted 0 items

[Iteration 0] Searching HTML...
  ✓ Found 10 sections
[Iteration 0] Generating code...
  ✓ Generated 977 chars of code
[Iteration 0] Executing code...
[]
  ✓ Extracted 0 items

[Iteration 0] Searching HTML...
  ✓ Found 10 secti

GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://docs.langchain.com/oss/python/langgraph/errors/GRAPH_RECURSION_LIMIT

## 15. Analysis: Why This Hybrid Approach Works

### The Problem with Pure Approaches

**Pure RAG:**
- ❌ Retrieves only K chunks (misses data)
- ❌ LLM context limits number of items
- ❌ No guarantee of completeness
- ✅ Good at finding WHERE data is

**Pure Code Generation:**
- ❌ Needs to know HTML structure upfront
- ❌ Blind to content location
- ✅ Extracts ALL matching items
- ✅ Deterministic and repeatable

### The Hybrid Solution

```
┌─────────────┐         ┌──────────────┐         ┌─────────────┐
│   Query     │────────▶│   Semantic   │────────▶│   Relevant  │
│             │         │    Search    │         │   Sections  │
└─────────────┘         └──────────────┘         └─────────────┘
                                                         │
                                                         ▼
┌─────────────┐         ┌──────────────┐         ┌─────────────┐
│  Complete   │◀────────│   Execute    │◀────────│  Generate   │
│    Data     │         │     Code     │         │    Code     │
└─────────────┘         └──────────────┘         └─────────────┘
       │                                                 │
       │                                                 │
       └─────────────────▶ Iterate if needed ◀──────────┘
```

### Key Advantages

1. **Semantic Understanding**: Search finds the right sections using embeddings
2. **Complete Extraction**: Code runs on FULL HTML, gets ALL items
3. **Iterative Refinement**: Agent can improve code based on results
4. **Deterministic**: Same structure → same code → same results
5. **Robust**: Handles edge cases with error handling

### When to Use Each Approach

| Scenario | Best Approach | Why |
|----------|--------------|-----|
| Small HTML (<10KB) | Direct LLM | No need for search, fits in context |
| Large HTML with lists | **Hybrid** | Search finds structure, code extracts all |
| Known HTML structure | Pure Code Gen | Skip search, write code directly |
| Question answering | Pure RAG | No need for completeness, just answer |
| Dynamic/varying sites | **Hybrid** | Search adapts, code generation is flexible |


In [18]:
@tool
def search_html_sections(query: str, num_results: int = 5) -> str:
    """
    Search HTML content semantically to find relevant sections.
    Use this to understand the HTML structure before generating extraction code.
    
    Args:
        query: What to search for (e.g., "job listings", "book prices")
        num_results: Number of relevant sections to return
    
    Returns:
        String with relevant HTML sections
    """
    if search_engine is None:
        return "Error: Search engine not initialized"
    
    results = search_engine.search(query, k=num_results)
    return "\n\n---\n\n".join(results[:3])  # Return top 3 to avoid overwhelming

@tool
def execute_extraction_code(python_code: str) -> str:
    """
    Execute BeautifulSoup extraction code on the full HTML.
    
    The code should:
    1. Parse HTML with: soup = BeautifulSoup(html_content, 'lxml')
    2. Extract data using BeautifulSoup methods
    3. Return a list of dictionaries with extracted data
    
    Args:
        python_code: BeautifulSoup extraction code
    
    Returns:
        JSON string with extracted data or error message
    """
    try:
        # Create safe execution environment
        local_vars = {
            'html_content': search_engine.html_content if search_engine else "",
            'BeautifulSoup': BeautifulSoup,
            'json': json
        }
        
        # Execute code
        exec(python_code, {}, local_vars)
        
        # Get result (code should set 'result' variable)
        result = local_vars.get('result', [])
        
        return json.dumps(result, indent=2)
    
    except Exception as e:
        return f"Error executing code: {str(e)}\n\nCode was:\n{python_code}"

@tool
def get_html_sample(num_chars: int = 8000) -> str:
    """
    Get a sample of the HTML to understand its structure.
    
    Args:
        num_chars: Number of characters to return
    
    Returns:
        HTML sample
    """
    if search_engine is None:
        return "Error: Search engine not initialized"
    
    return search_engine.html_content[:num_chars]

# Collect all tools
tools = [search_html_sections, execute_extraction_code, get_html_sample]

print(f"✓ Defined {len(tools)} tools for the agent")

✓ Defined 3 tools for the agent


## 6. Create Agent

ReAct-style agent that can iteratively search and generate code

In [19]:
# Create agent prompt
agent_prompt = ChatPromptTemplate.from_messages([
    ("system", """
You are an expert at extracting structured data from HTML.

Your approach:
1. Use search_html_sections to find where the relevant data is located
2. Analyze the HTML structure from search results
3. Generate BeautifulSoup extraction code using execute_extraction_code
4. If results are incomplete, iterate:
   - Search for different patterns
   - Refine your extraction code
   - Try different selectors
5. Return complete, structured JSON data

Code generation rules:
- Always use: soup = BeautifulSoup(html_content, 'lxml')
- Store results in a variable called 'result' (list of dicts)
- Extract ALL matching items, not just a sample
- Use robust selectors (class names, tag structure)
- Handle missing data gracefully

Example code structure:
```python
soup = BeautifulSoup(html_content, 'lxml')
result = []
items = soup.find_all('div', class_='item')
for item in items:
    result.append({
        'name': item.find('h2').text.strip(),
        'price': item.find('span', class_='price').text.strip()
    })
```
    """),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad")
])

# Note: create_tool_calling_agent requires a model that supports tool calling
# For Ollama, we'll use a simpler approach with structured prompting
print("✓ Agent prompt created")

✓ Agent prompt created


## 7. Simple Agent Implementation

Since Ollama doesn't natively support tool calling, let's create a simpler agent loop

In [20]:
def hybrid_extraction_agent(
    html_file_path: Path,
    query: str,
    max_iterations: int = 3
) -> Dict[str, Any]:
    """
    Hybrid extraction: Search semantically, then generate code
    
    Args:
        html_file_path: Path to HTML file
        query: Extraction query
        max_iterations: Max attempts to refine
    
    Returns:
        Extracted data
    """
    global search_engine
    
    print(f"\n{'='*80}")
    print(f"Processing: {html_file_path.name}")
    print(f"Query: {query}")
    print(f"{'='*80}\n")
    
    # Load HTML
    with open(html_file_path, 'r', encoding='utf-8') as f:
        html_content = f.read()
    
    # Initialize search engine
    print("Step 1: Building semantic search index...")
    search_engine = HTMLSearchEngine(html_content, embeddings)
    print("  ✓ Index built\n")
    
    # Step 2: Search for relevant sections
    print("Step 2: Searching for relevant HTML sections...")
    relevant_sections = search_html_sections.invoke({"query": query, "num_results": 20})
    print(f"  ✓ Found relevant sections ({len(relevant_sections)} chars)\n")
    print(f"Sample:\n{relevant_sections[:500]}...\n")
    
    # Step 3: Generate extraction code
    print("Step 3: Generating BeautifulSoup extraction code...")
    
    code_gen_prompt = f"""
Based on this HTML sample and query, generate BeautifulSoup extraction code.

Query: {query}

HTML Sample:
{relevant_sections[:8000]}

Generate Python code that:
1. Parses HTML: soup = BeautifulSoup(html_content, 'lxml')
2. Finds ALL matching elements (not just one)
3. Extracts the requested fields
4. Stores results in: result = [list of dicts]

Return ONLY the Python code, no explanations.
Code:
"""
    
    code_response = llm.invoke(code_gen_prompt)
    
    # Extract code from response
    code = code_response
    if "```python" in code:
        code = code.split("```python")[1].split("```")[0]
    elif "```" in code:
        code = code.split("```")[1].split("```")[0]
    
    print(f"  ✓ Generated code ({len(code)} chars)\n")
    print(f"Code:\n{code}\n")
    
    # Step 4: Execute code
    print("Step 4: Executing extraction code...")
    result_json = execute_extraction_code.invoke({"python_code": code})
    
    try:
        result = json.loads(result_json)
        print(f"  ✓ Extracted {len(result) if isinstance(result, list) else 'some'} items\n")
        return result
    except json.JSONDecodeError:
        print(f"  ✗ Error: {result_json}\n")
        return {"error": result_json}

print("✓ Hybrid agent function defined")

✓ Hybrid agent function defined


## 8. Test on All Scenarios

In [21]:
# Test hybrid approach on all scenarios
hybrid_results = {}

for scenario_name, html_file in HTML_FILES.items():
    query = TEST_QUERIES[scenario_name]
    
    try:
        result = hybrid_extraction_agent(html_file, query)
        hybrid_results[scenario_name] = result
        
        print("\n" + "="*80)
        print("RESULT:")
        if isinstance(result, list):
            print(f"✓ Extracted {len(result)} items")
            print("\nFirst 2 items:")
            print(json.dumps(result[:2], indent=2))
        else:
            print(json.dumps(result, indent=2))
        print("="*80 + "\n\n")
        
    except Exception as e:
        print(f"\n✗ Error: {e}\n")
        import traceback
        traceback.print_exc()
        hybrid_results[scenario_name] = {"error": str(e)}


Processing: scenario1_books.html
Query: Can you return me the books: name and price?

Step 1: Building semantic search index...
  Building hybrid index (clean text embeddings + raw HTML storage)...
  ✓ Indexed 3 chunks (clean text → raw HTML mapping)
  ✓ Index built

Step 2: Searching for relevant HTML sections...
  ✓ Found relevant sections (84614 chars)

Sample:
<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html lang="en-us" class="no-js"> <!--<![endif]-->
    <head>
        <title>
    All products | Books to Scrape - Sandbox
</title>

        <meta http-equiv="content-type" content="text/html; charset=UTF-8" /...

Step 3: Generating BeautifulSoup extraction code...
  ✓ Generated code (365 chars)

Code:

from bs4 import BeautifulSoup

html_

## 9. Results Summary

In [ ]:
print("\n" + "="*80)
print("HYBRID APPROACH RESULTS: Semantic Search + Code Generation")
print("="*80 + "\n")

for scenario_name, result in hybrid_results.items():
    print(f"{scenario_name}:")
    if isinstance(result, list):
        print(f"  ✓ Extracted {len(result)} items")
        if len(result) > 0:
            print(f"  Fields: {', '.join(result[0].keys())}")
    elif isinstance(result, dict) and "error" in result:
        print(f"  ✗ Error: {result['error'][:100]}")
    else:
        print(f"  ⚠️  Partial result")
    print()

print("="*80)
print("\n🎯 KEY ADVANTAGES OF THIS APPROACH:\n")
print("1. Search finds WHERE data is (leverages semantic understanding)")
print("2. Code generation extracts WHAT data is (systematic, complete)")
print("3. Runs on FULL HTML (no chunking limitations)")
print("4. Deterministic extraction (same code → same results)")
print("5. Iterative refinement possible (agent can improve code)")
print("\n" + "="*80)

## 10. Comparison with Pure RAG

In [ ]:
print("\n📊 APPROACH COMPARISON:\n")
print("Approach            | Search | Extraction | Complete | Fast | Deterministic")
print("-" * 80)
print("Pure RAG            |   ✅   |     ❌     |    ❌    |  ✅  |      ❌")
print("Pure Code Gen       |   ❌   |     ✅     |    ✅    |  ✅  |      ✅")
print("HYBRID (This)       |   ✅   |     ✅     |    ✅    |  ⚠️   |      ✅")
print("-" * 80)
print("\n✅ Pure RAG: Good at finding info, bad at complete extraction")
print("✅ Pure Code Gen: Great extraction, but needs to know where to look")
print("🎯 HYBRID: Best of both - search finds it, code extracts it!")

## 11. Next Steps & Improvements

### Possible Enhancements:

1. **Better Agent Framework**
   - Use LangGraph for more sophisticated agent loops
   - Implement proper ReAct pattern with tool calling
   - Add memory for multi-turn refinement

2. **Code Validation**
   - Test generated code on HTML sample first
   - Validate results meet expected schema
   - Auto-retry with feedback if extraction fails

3. **Smarter Search**
   - Use query to determine search strategy
   - Adjust k based on expected result count
   - Try multiple search queries if needed

4. **Code Library**
   - Cache successful extraction patterns
   - Build library of common HTML structures
   - Reuse patterns for similar sites

5. **Production Features**
   - Add rate limiting for LLM calls
   - Implement proper error handling
   - Log agent reasoning for debugging
   - Add metrics (extraction accuracy, speed)